# NLP Critique de film

## Importation des bibliothèques

In [1]:
import requests
import lxml.html as lh
import pandas as pd

## Scrapping

In [2]:
page = requests.get("https://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/")
doc = lh.fromstring(page.content)

com = doc.xpath('//div[@class="content-txt review-card-content"]')
note = doc.xpath('//span[@class="stareval-note"]')

In [3]:
# for i in range(len(note)):
#     print(note[i].text_content().strip() + "\n" + \
#           com[i].text_content().strip() + "\n")

In [9]:
# URL
url = "https://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/"
# URI-SUFFIX
uri_pages = '?page='
# Number of pages to read
nbPages = 100
# XPath content to collect
tags = ['//span[@class="stareval-note"]', \
        '//div[@class="content-txt review-card-content"]']

cols = ["Note", \
        "Commentaire"]

urls = ["https://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/", \
        "https://www.allocine.fr/film/fichefilm-281203/critiques/spectateurs/"]

In [5]:
page = requests.get(url)
doc = lh.fromstring(page.content)

In [10]:
def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)

    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))

    # Gather the data into a Pandas DataFrame array
    df_liste = []
    for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)

    # Build the unique Dataframe with one tag (xpath) content per column
    liste = df_liste[0]
    for j in range(len(tags)-1):
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
    
    return liste

def getPages(_nbPages, _urls):
    liste_finale = pd.DataFrame()
    for _url in _urls:
        for i in range (_nbPages):
            liste = getPage(_url + uri_pages + str(i+1))
            liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
        print(liste_finale.shape)
    return liste_finale

In [11]:
getPage(url).shape

(15, 3)

In [26]:
list_totale = getPages(nbPages, urls)

(1500, 3)
(2810, 3)


In [27]:
list_totale

,Note,Commentaire,key
0,"5,0",Après le chef d'oeuvre super-héroïque The Dark...,0
1,"5,0",C’est fou ce qu’on aime détester Christopher N...,1
2,"5,0",CHEF D’ŒUVRE ! Le film est absolument parfait ...,2
3,"5,0","Un film aussi novateur que complexe, dont la m...",3
4,"3,5",Le meilleur blockbuster de 2010 a pour thème l...,4
...,...,...,...
2805,"3,0","léger et sans prise de tête, le film permet de...",8
2806,"0,5",Films vraiment pour les enfants je me suis fai...,9
2807,"5,0","Ce film etait juste ouffissime , j'ai vraiment...",10
2808,"1,0","Déconseillé aux enfants de plus de 7 ans, et e...",11


## Préparation des données

In [88]:
df = list_totale.copy()

In [89]:
df["Commentaire"] = df["Commentaire"].replace("[^\w\s]", " ", regex=True).str.replace('\d+', '', regex=True).str.lower()
df.head()

,Note,Commentaire,key
0,"5,0",après le chef d oeuvre super héroïque the dark...,0
1,"5,0",c est fou ce qu on aime détester christopher n...,1
2,"5,0",chef d œuvre le film est absolument parfait ...,2
3,"5,0",un film aussi novateur que complexe dont la m...,3
4,"3,5",le meilleur blockbuster de a pour thème la dé...,4


### Soustraction de stopwords 

In [90]:
from stop_words import get_stop_words

In [91]:
stopwords = get_stop_words('french')
print(stopwords)

['a', 'ai', 'aie', 'aient', 'aies', 'ait', 'alors', 'as', 'au', 'aucun', 'aura', 'aurai', 'auraient', 'aurais', 'aurait', 'auras', 'aurez', 'auriez', 'aurions', 'aurons', 'auront', 'aussi', 'autre', 'aux', 'avaient', 'avais', 'avait', 'avant', 'avec', 'avez', 'aviez', 'avions', 'avoir', 'avons', 'ayant', 'ayez', 'ayons', 'bon', 'car', 'ce', 'ceci', 'cela', 'ces', 'cet', 'cette', 'ceux', 'chaque', 'ci', 'comme', 'comment', 'd', 'dans', 'de', 'dedans', 'dehors', 'depuis', 'des', 'deux', 'devoir', 'devrait', 'devrez', 'devriez', 'devrions', 'devrons', 'devront', 'dois', 'doit', 'donc', 'dos', 'droite', 'du', 'dès', 'début', 'dù', 'elle', 'elles', 'en', 'encore', 'es', 'est', 'et', 'eu', 'eue', 'eues', 'eurent', 'eus', 'eusse', 'eussent', 'eusses', 'eussiez', 'eussions', 'eut', 'eux', 'eûmes', 'eût', 'eûtes', 'faire', 'fais', 'faisez', 'fait', 'faites', 'fois', 'font', 'force', 'furent', 'fus', 'fusse', 'fussent', 'fusses', 'fussiez', 'fussions', 'fut', 'fûmes', 'fût', 'fûtes', 'haut', 'ho

In [92]:
stopwords.append("c")

In [93]:
df['sans_sw'] = df['Commentaire'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
df.head()

,Note,Commentaire,key,sans_sw
0,"5,0",après le chef d oeuvre super héroïque the dark...,0,après chef oeuvre super héroïque the dark knig...
1,"5,0",c est fou ce qu on aime détester christopher n...,1,fou aime détester christopher nolan plus films...
2,"5,0",chef d œuvre le film est absolument parfait ...,2,chef œuvre film absolument parfait acteurs exc...
3,"5,0",un film aussi novateur que complexe dont la m...,3,film novateur complexe dont mise scène touche ...
4,"3,5",le meilleur blockbuster de a pour thème la dé...,4,meilleur blockbuster thème délicate différenci...


### Etape de LEM

In [76]:
import spacy

In [87]:
nlp = spacy.load('fr_core_news_md')

doc = nlp(u"voudrais était non animaux yeux dors couvre.")
for token in doc:
    print(token, token.lemma_)

voudrais vouloir
était être
non non
animaux animal
yeux yeux
dors dors
couvre couvrir
. .


In [111]:
# df["lem"] = df["sans_sw"].apply(lambda x: [y.lemma_ for y in nlp(x)])
# df.head()

,Note,Commentaire,key,sans_sw,lem
0,"5,0",après le chef d oeuvre super héroïque the dark...,0,après chef oeuvre super héroïque the dark knig...,"[après, chef, oeuvre, super, héroïque, the, da..."
1,"5,0",c est fou ce qu on aime détester christopher n...,1,fou aime détester christopher nolan plus films...,"[fou, aime, détester, christopher, nolan, plus..."
2,"5,0",chef d œuvre le film est absolument parfait ...,2,chef œuvre film absolument parfait acteurs exc...,"[chef, œuvre, film, absolument, parfait, acteu..."
3,"5,0",un film aussi novateur que complexe dont la m...,3,film novateur complexe dont mise scène touche ...,"[film, novateur, complexe, dont, mettre, scène..."
4,"3,5",le meilleur blockbuster de a pour thème la dé...,4,meilleur blockbuster thème délicate différenci...,"[meilleur, blockbuster, thème, délicat, différ..."


In [113]:
df['lem'] = df["sans_sw"].apply(lambda x: ' '.join(str(y.lemma_) for y in nlp(x)))
df.head()

,Note,Commentaire,key,sans_sw,lem
0,"5,0",après le chef d oeuvre super héroïque the dark...,0,après chef oeuvre super héroïque the dark knig...,après chef oeuvre super héroïque the dark knig...
1,"5,0",c est fou ce qu on aime détester christopher n...,1,fou aime détester christopher nolan plus films...,fou aime détester christopher nolan plus film ...
2,"5,0",chef d œuvre le film est absolument parfait ...,2,chef œuvre film absolument parfait acteurs exc...,chef œuvre film absolument parfait acteur exce...
3,"5,0",un film aussi novateur que complexe dont la m...,3,film novateur complexe dont mise scène touche ...,film novateur complexe dont mettre scène touch...
4,"3,5",le meilleur blockbuster de a pour thème la dé...,4,meilleur blockbuster thème délicate différenci...,meilleur blockbuster thème délicat différencia...


### Retrait des accents

In [115]:
df["lem"] = df["lem"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df.head()

,Note,Commentaire,key,sans_sw,lem
0,"5,0",après le chef d oeuvre super héroïque the dark...,0,apres chef oeuvre super heroique the dark knig...,apres chef oeuvre super heroique the dark knig...
1,"5,0",c est fou ce qu on aime détester christopher n...,1,fou aime detester christopher nolan plus films...,fou aime detester christopher nolan plus film ...
2,"5,0",chef d œuvre le film est absolument parfait ...,2,chef uvre film absolument parfait acteurs exce...,chef uvre film absolument parfait acteur excel...
3,"5,0",un film aussi novateur que complexe dont la m...,3,film novateur complexe dont mise scene touche ...,film novateur complexe dont mettre scene touch...
4,"3,5",le meilleur blockbuster de a pour thème la dé...,4,meilleur blockbuster theme delicate differenci...,meilleur blockbuster theme delicat differencia...


#### Si liste de mots

In [100]:
# import unicodedata as ud

In [101]:
# df["lem"] = df["lem"].apply(lambda x: [ud.normalize('NFKD',y).encode('ascii', errors='ignore').decode('utf-8') for y in x])
# df.head()

,Note,Commentaire,key,sans_sw,lem
0,"5,0",après le chef d oeuvre super héroïque the dark...,0,après chef oeuvre super héroïque the dark knig...,"[apres, chef, oeuvre, super, heroique, the, da..."
1,"5,0",c est fou ce qu on aime détester christopher n...,1,fou aime détester christopher nolan plus films...,"[fou, aime, detester, christopher, nolan, plus..."
2,"5,0",chef d œuvre le film est absolument parfait ...,2,chef œuvre film absolument parfait acteurs exc...,"[chef, uvre, film, absolument, parfait, acteur..."
3,"5,0",un film aussi novateur que complexe dont la m...,3,film novateur complexe dont mise scène touche ...,"[film, novateur, complexe, dont, mettre, scene..."
4,"3,5",le meilleur blockbuster de a pour thème la dé...,4,meilleur blockbuster thème délicate différenci...,"[meilleur, blockbuster, theme, delicat, differ..."


### Etape de *STEM*

In [69]:
# from nltk.stem.snowball import SnowballStemmer

In [73]:
# stemmer = SnowballStemmer("french")

In [74]:
# df["stemmed"] = df["sans_sw"].str.split().apply(lambda x: [stemmer.stem(y) for y in x])
# df.head()

,Note,Commentaire,key,sans_sw,stemmed
0,"5,0",après le chef d oeuvre super héroïque the dark...,0,apres chef oeuvre super heroique the dark knig...,"[apre, chef, oeuvr, sup, heroiqu, the, dark, k..."
1,"5,0",c est fou ce qu on aime détester christopher n...,1,fou aime detester christopher nolan plus films...,"[fou, aim, detest, christoph, nolan, plus, fil..."
2,"5,0",chef d œuvre le film est absolument parfait ...,2,chef uvre film absolument parfait acteurs exce...,"[chef, uvre, film, absolu, parf, acteur, excel..."
3,"5,0",un film aussi novateur que complexe dont la m...,3,film novateur complexe dont mise scene touche ...,"[film, novateur, complex, dont, mis, scen, tou..."
4,"3,5",le meilleur blockbuster de a pour thème la dé...,4,meilleur blockbuster theme delicate differenci...,"[meilleur, blockbust, them, delicat, differenc..."


### Transformation en *Bag of words*

In [102]:
from sklearn.feature_extraction.text import CountVectorizer

In [117]:
cv = CountVectorizer(lowercase=False)
cv_matrix = cv.fit_transform(df["lem"])

print(cv_matrix.shape, "\n", type(cv_matrix))

(2810, 8642) 
 <class 'scipy.sparse.csr.csr_matrix'>


In [118]:
df_bow = pd.DataFrame(cv_matrix.toarray(), index = df['key'].values, columns = cv.get_feature_names())
df_bow.head()

,Monsieur,aahahahahah,aakhen,aaron,abandon,abandonner,abasourdir,abattage,abattre,abbuse,...,your,yusuf,zebrer,zen,zero,zest,zeu,zimmer,zizanie,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
